In [1]:
from model import MLP

model = MLP()

print("LinW layers:", "\n".join([f"Depth {model[i].depth}: {model[i]}" for i in range(len(model))]), sep="\n\n")

LinW layers:

Depth 0: LinW(in_features=512, out_features=512, bias=True)
Depth 1: LinW(in_features=512, out_features=512, bias=True)


In [2]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss /= len(train_loader.dataset)
    accuracy = 100. * correct / total
    return train_loss, accuracy

In [3]:
import torch

def evaluate(device, model, dataloader):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

EPOCHS = 10
BATCH_SIZE = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = MLP().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_accuracy = evaluate(device, model, test_loader)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}%, Test accuracy: {test_accuracy:.2f}%')
    lr_scheduler.step()


Files already downloaded and verified
Files already downloaded and verified
tensor([[-0.0252,  0.0389, -0.0153,  ..., -0.0221, -0.0163,  0.0267],
        [ 0.0075, -0.0127,  0.0221,  ...,  0.0166,  0.0334,  0.0027],
        [ 0.0190,  0.0310,  0.0333,  ..., -0.0172,  0.0219,  0.0394],
        ...,
        [ 0.0197,  0.0272,  0.0143,  ...,  0.0177,  0.0319,  0.0295],
        [ 0.0076,  0.0232,  0.0120,  ..., -0.0393, -0.0066, -0.0242],
        [ 0.0180, -0.0377, -0.0069,  ...,  0.0134,  0.0161,  0.0230]],
       device='cuda:0', grad_fn=<MulBackward0>)
tensor([[-0.0292,  0.0032,  0.0266,  ..., -0.0008, -0.0342,  0.0284],
        [-0.0104, -0.0121,  0.0149,  ...,  0.0307,  0.0085, -0.0316],
        [-0.0339, -0.0023, -0.0264,  ...,  0.0087, -0.0081, -0.0214],
        ...,
        [ 0.0208, -0.0239,  0.0045,  ...,  0.0115,  0.0228, -0.0334],
        [-0.0163, -0.0330,  0.0062,  ..., -0.0098,  0.0306,  0.0186],
        [-0.0211, -0.0295,  0.0336,  ...,  0.0334,  0.0104,  0.0181]],
       d

KeyboardInterrupt: 